## Label and Transform content locally
This notebook walks through training a transformation model and redacting PII locally in your environment.

Follow the instructions here to set up your local environment: https://docs.gretel.ai/environment-setup

Prerequisites:
* Python 3.9+ (`python --version`).
* Ensure that Docker is running (`docker info`).
* The Gretel client SDK is installed and configured (`pip install -U gretel-client; gretel configure`).

In [ ]:
import json

import yaml
from smart_open import open
import pandas as pd

from gretel_client import create_project, submit_docker_local

data_source = "https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/example-datasets/bike-customer-orders.csv"

# Simple policy to redact PII types with a character. 
# Dates are shifted +/- 20 days based on the CustomerID field.
# Income is bucketized to 5000 number increments.

config = """
schema_version: 1.0
models:
  - transforms:
      data_source: "_"
      policies:
        - name: remove_pii
          rules:
            - name: fake_or_redact_pii
              conditions:
                value_label:
                  - person_name
                  - phone_number
                  - gender
                  - birth_date
              transforms:
                - type: redact_with_char
                  attrs:
                    char: X
            - name: dateshifter
              conditions:
                field_label:
                  - date
                  - datetime
                  - birth_date
              transforms:
                - type: dateshift
                  attrs:
                    min: 20
                    max: 20
                    formats: "%Y-%m-%d"
                    field_name: "CustomerID"        
            - name: bucketize-income
              conditions:
                field_name:
                  - YearlyIncome
              transforms:
                - type: numberbucket
                  attrs:
                    min: 0
                    max: 1000000
                    nearest: 5000
"""


In [ ]:
# Load and preview the DataFrame to train the transform model on.

df = pd.read_csv(data_source, nrows=500)
df.to_csv('training_data.csv', index=False)
df.head(5)

In [ ]:
project = create_project()

In [ ]:
# The following cell will create the transform model and 
# run a sample of the data set through the model. this sample
# can be used to ensure the model is functioning correctly
# before continuing.
transform = project.create_model_obj(
    model_config=yaml.safe_load(config),
    data_source='training_data.csv'
)

run = submit_docker_local(transform, output_dir="tmp/")

In [ ]:
# Review the sampled classification report
# to get an overview of detected data types
report = json.loads(open("tmp/report_json.json.gz").read())
pd.DataFrame(report["metadata"]["fields"])

In [ ]:
# Next let's transform the remaining records using the transformation
# policy and model that was just created.
transform_records = transform.create_record_handler_obj(data_source='training_data.csv')

run = submit_docker_local(
    transform_records,
    model_path="tmp/model.tar.gz",
    output_dir="tmp/"
)

In [ ]:
# View the transformation report
report = json.loads(open("tmp/report_json.json.gz").read())
pd.DataFrame(report["metadata"]["fields"])

In [ ]:
# View the transformed data
results = pd.read_csv('tmp/data.gz')
results.head(5)